# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf #yahoo finances
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-04-19'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2660,2023-04-14,"$30,409.56","$31,005.61","$30,044.50","$30,485.70","$30,485.70",22659995079
2661,2023-04-15,"$30,490.75","$30,601.74","$30,245.88","$30,318.50","$30,318.50",11940685378
2662,2023-04-16,"$30,315.98","$30,555.54","$30,157.83","$30,315.36","$30,315.36",12854816417
2663,2023-04-17,"$30,317.15","$30,319.20","$29,275.37","$29,445.04","$29,445.04",17872186762
2664,2023-04-18,"$29,449.09","$30,470.30","$29,154.85","$30,397.55","$30,397.55",19480529496


In [ ]:
# Validamos que no hay datos vacíos
btc_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[["Date","Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df

df.rename(columns=new_names, inplace=True)
df.head()

,ds,y
0,2016-01-01,$430.72
1,2016-01-02,$434.62
2,2016-01-03,$433.58
3,2016-01-04,$430.06
4,2016-01-05,$433.07


In [ ]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
2660,2023-04-14,"$30,409.56"
2661,2023-04-15,"$30,490.75"
2662,2023-04-16,"$30,315.98"
2663,2023-04-17,"$30,317.15"
2664,2023-04-18,"$29,449.09"


In [ ]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = ""
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [ ]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m =Prophet(seasonality_mode = 'multiplicative')

In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future =  m.make_future_dataframe(periods=365) #por defecto periods es dias, pero se puede cambiar por meses,etc.
future.tail()

,ds
3025,2024-04-13
3026,2024-04-14
3027,2024-04-15
3028,2024-04-16
3029,2024-04-17


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05
...,...
3025,2024-04-13
3026,2024-04-14
3027,2024-04-15
3028,2024-04-16


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
3025,2024-04-13,"$-12,406.30","$-31,306.32","$6,560.04"
3026,2024-04-14,"$-12,416.09","$-31,541.74","$6,870.60"
3027,2024-04-15,"$-12,427.91","$-30,525.52","$6,069.24"
3028,2024-04-16,"$-12,435.46","$-30,861.07","$5,868.56"
3029,2024-04-17,"$-12,417.49","$-30,856.69","$5,256.04"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)